Изобретать велосипед стоит только в том случае, когда хочешь разобраться, как он работает.

Если же поставлена задача для решения, первое действие - поиск готовых вариантов.

---
### Загрузка данных для предикта и функции общего назначения

In [5]:
import pandas as pd


# read_csv ломается, так как в строках переменное число запятых
rows = list()
with open("dataset_1937770_3.txt", encoding="utf-8", mode='r') as file:
    header = next(file).rstrip('\n').split(',') 
    for line in file:
        line = line.rstrip('\n')
        if not line:
            continue
        id_str, text_no_space = line.split(',', maxsplit=1) # только первая запятая
        rows.append((int(id_str), text_no_space))

df = pd.DataFrame(data=rows, columns=header)
df

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


In [6]:
def find_spaces(text: str) -> list[int]:
    '''
    Поиск индексов пробелов

    На вход подаётся строка с восстановленными пробелами
    '''
    spaces = list()
    accumed_text = 0 # накопившийся обрезанный текст для правильной индексации
    accumed_spaces = 0 # накопившиеся пробелы для приведения к слитной строке
    while True:
        space_idx = text.find(" ")
        if space_idx == -1: # условие выхода - ненаход
            break
        # логика подсказана здравым смыслом и опытом, тесты проходит
        spaces.append(accumed_text + space_idx - accumed_spaces)

        text = text[space_idx+1:] # +1 для пропуска найденного пробела

        accumed_text += space_idx + 1 # см. коммент выше
        accumed_spaces += 1 # очев
    return spaces


def make_submission(df, restored: pd.Series):
    '''
    restored - pd.Series из строк с восстановленными пробелами
    '''

    df["predicted_positions"] = restored.apply(find_spaces).apply(str)
    res = df.drop(columns=("text_no_spaces"))
    res.to_csv("submission.csv", index=False)

In [7]:
# тесты find_spaces

text = "ищу дом в Подмосковье" # ищудомвПодмосковье [3, 6, 7]
text = "Весна - я уже не грею пио." # Весна-яуженегреюпио. [5, 6, 7, 10, 12, 16]
print(find_spaces(text)) 

[5, 6, 7, 10, 12, 16]


---
### Экспериментальная секция

In [8]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")

# text = "ищудомвПодмосковье"

# tokens = tokenizer.tokenize(text)
# restored = str.join(" ", tokens)

# print(tokens)
# print(restored)

for text in df.text_no_spaces:
    tokens = tokenizer.tokenize(text)
    tokens = [t.strip("#") for t in tokens]
    restored = str.join(" ", tokens)
    print(restored)

куплю а и фон 14 про
ищу дом в под москов ье
сда юк варти рус ме белью ите х нико и
нов ы ид ив ан дос тав кане доро го
отдам даром ко шку
работа в мос к ве уда ленно
куплю теле визор ph ili ps
ищу груз чиков для пере езда
ремонт кварти р под клю ч
куплю ноу т бук h p
ищу кварти ру уме тро
новая мик рово лн ов ка s amsung
срочно прода м вело сипе д
куплю гита ру fe nd er
ищу ре петит ора по био логии
сда ю гара жна дли тельны ис рок
куплю дива н бу
ищу мастер апо ремонт у хол о диль ников
нов ы и шка ф дос тав кас его дня
куплю x box one
ищу под рабо тку пов е чера м
сдам комна тус ту дент ке
куплю стар ую кни гу
ищу соб ак ула бра дор
нов ы ите ле фон x iaomi 13
куплю play st ation 5 дис к
ищу комна ту в центре города
срочно ну жна нян яр е бенку
куплю сти раль ную маш ину ind es it
ищу дет скую кров ат ку бу
новая кур тка дос тав ка
куплю вело сипе д mer ida
ищу вра ча оф таль моло га
сда юк варти ру в центре мос к вы
куплю хол о диль ник s amsung
ищу ко шку бри тан скую
нов ы ино ут

In [9]:
# первый собранный рабочий вариант с Mean F1 = 49.502%

from transformers import AutoTokenizer


# tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru") 

tokenized = df['text_no_spaces'].apply(tokenizer.tokenize) # токенизация 
tokenized = tokenized.apply(lambda l: [s.strip("#") for s in l]) # удаление "##" перед токенами
restored = tokenized.apply(lambda l: str.join(' ', l)) # восстановление строк

make_submission(df, restored)

# проблема: слишком агрессивное разделение на токены
# можно попробовать другие варианты

In [10]:
from razdel import tokenize


def restore_with_razdel(text: str):
    tokens = [t.text for t in tokenize(text)]
    return " ".join(tokens)


for text in df.text_no_spaces:
    restored = restore_with_razdel(text)
    print(restored)

# этот напротив не разделяет практически вообще русские слова
# хорошо отделяет числа и английские слова, можно использовать как препроцессор

куплюайфон 14 про
ищудомвПодмосковье
сдаюквартирусмебельюитехникой
новыйдивандоставканедорого
отдамдаромкошку
работавМосквеудаленно
куплютелевизор Philips
ищугрузчиковдляпереезда
ремонтквартирподключ
куплюноутбук HP
ищуквартирууметро
новаямикроволновка Samsung
срочнопродамвелосипед
куплюгитару Fender
ищурепетиторапобиологии
сдаюгаражнадлительныйсрок
куплюдиванбу
ищумастерапоремонтухолодильников
новыйшкафдоставкасегодня
куплю XboxOne
ищуподработкуповечерам
сдамкомнатустудентке
куплюстаруюкнигу
ищусобакулабрадор
новыйтелефон Xiaomi 13
куплю Playstation 5 диск
ищукомнатувцентрегорода
срочнонужнаняняребенку
куплюстиральнуюмашину Indesit
ищудетскуюкроваткубу
новаякурткадоставка
куплювелосипед Merida
ищуврачаофтальмолога
сдаюквартирувцентреМосквы
куплюхолодильник Samsung
ищукошкубританскую
новыйноутбукдоставказавтра
куплюшкафбу
ищурепетиторапоистории
сдаюкомнатубезпосредников
куплютелевизор LGOLED
ищуквартирунедорогосрочно
новыйайфондоставка
куплюгитару Yamaha
ищустудиюуметро
сдамквартирумол

In [11]:
# https://github.com/Koziev/rutokenizer

import rutokenizer


def restore_with_rutokenizer(text: str, tokenizer: rutokenizer.Tokenizer):
    tokens = tokenizer.tokenize(text)
    return " ".join(tokens)

tokenizer = rutokenizer.Tokenizer()
tokenizer.load()

for text in df.text_no_spaces:
    restored = restore_with_rutokenizer(text, tokenizer)
    print(restored)

# бесполезен

куплюайфон14про
ищудомвПодмосковье
сдаюквартирусмебельюитехникой
новыйдивандоставканедорого
отдамдаромкошку
работавМосквеудаленно
куплютелевизорPhilips
ищугрузчиковдляпереезда
ремонтквартирподключ
куплюноутбукHP
ищуквартирууметро
новаямикроволновкаSamsung
срочнопродамвелосипед
куплюгитаруFender
ищурепетиторапобиологии
сдаюгаражнадлительныйсрок
куплюдиванбу
ищумастерапоремонтухолодильников
новыйшкафдоставкасегодня
куплюXboxOne
ищуподработкуповечерам
сдамкомнатустудентке
куплюстаруюкнигу
ищусобакулабрадор
новыйтелефонXiaomi13
куплюPlaystation5диск
ищукомнатувцентрегорода
срочнонужнаняняребенку
куплюстиральнуюмашинуIndesit
ищудетскуюкроваткубу
новаякурткадоставка
куплювелосипедMerida
ищуврачаофтальмолога
сдаюквартирувцентреМосквы
куплюхолодильникSamsung
ищукошкубританскую
новыйноутбукдоставказавтра
куплюшкафбу
ищурепетиторапоистории
сдаюкомнатубезпосредников
куплютелевизорLGOLED
ищуквартирунедорогосрочно
новыйайфондоставка
куплюгитаруYamaha
ищустудиюуметро
сдамквартирумолодоженам
куплюком

In [12]:
# подготовка данных для обучения sentencepiece

with open("train.txt", "w", encoding='utf-8') as train:
    for line in df["text_no_spaces"]:
        # line = restore_with_razdel(line) # можно подключить razdel как препроцессор
        train.write(line + "\n")

# с razdel'ом на глаз результат хуже, после препроцессинга spm начинает дробить английские слова

In [13]:
import sentencepiece as spm

# обучение модели
spm.SentencePieceTrainer.train(
    input='train.txt', # файл, где каждая строка = один пример текста
    model_prefix='mymodel',
    vocab_size=8000,
    character_coverage=1.0, # важно для кириллицы, чтобы не потерять буквы
    model_type='bpe' # можно unigram попробовать как вариант
)

sp = spm.SentencePieceProcessor()
sp.load("mymodel.model")

for text in df["text_no_spaces"]:
    tokens = sp.encode(text, out_type=str)
    print(tokens)

# показывает сбалансированное разделение даже на сырых тренировочных данных
# не такой агрессивный, как Bert'ы, отделяет иногда неплохо
# Mean F1 = 36.254%, всё ещё хуже самого первого решения
# по-хорошему, нужно ещё что-то вроде пост-обработки


# def restore_with_spm(text: str):
#     tokens = sp.encode(text, out_type=str)
#     tokens = [t.strip("▁") for t in tokens]
#     return str.join(" ", tokens)

# restored = df["text_no_spaces"].apply(restore_with_spm)
# make_submission(df, restored)

['▁куплюайфон', '14', 'про']
['▁ищудом', 'вПодмосковье']
['▁сдаюквартиру', 'смебелью', 'итехникой']
['▁новыйдиван', 'доставканедорого']
['▁отдамдаромкошку']
['▁работавМоскве', 'удаленно']
['▁куплютелевизор', 'Philips']
['▁ищугрузчи', 'ковдляпереезда']
['▁ремонт', 'квартирподключ']
['▁куплюноутбук', 'HP']
['▁ищуквартиру', 'уметро']
['▁новая', 'микроволновка', 'Samsung']
['▁срочнопродам', 'велосипед']
['▁куплюгитару', 'Fender']
['▁ищурепетиторапо', 'биологии']
['▁сдаюгараж', 'надлительныйсрок']
['▁куплюдиванбу']
['▁ищумастерапо', 'ремонту', 'холодильников']
['▁новыйшкаф', 'доставкасегодня']
['▁куплю', 'XboxOne']
['▁ищуподработку', 'повечерам']
['▁сдамкомнату', 'студентке']
['▁куплю', 'старуюкнигу']
['▁ищусобакула', 'брадор']
['▁новыйтелефон', 'Xiaomi', '13']
['▁куплю', 'Playstation', '5', 'диск']
['▁ищукомнату', 'вцентрегорода']
['▁срочно', 'нужнаняняребенку']
['▁куплюстираль', 'нуюмашину', 'Indesit']
['▁ищудетскую', 'кроваткубу']
['▁новаякуртка', 'доставка']
['▁куплювелосипед', 'Merida'

---
### Лучшее решение + препроцессинг + пост-обработка

Идея: взять как препроцессор `razdel`, далее сегментацию производить с помощью `AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")` по каждому токену, если он не является английским словом или числом, затем с тем же условием решать, склеивать токены обратно или нет, ориентируясь на внешний словарь.

Словарь взят отсюда: https://github.com/caffidev/russianwords/blob/main/utf-8/words_cases.txt

In [14]:
import re
from transformers import AutoTokenizer
from razdel import tokenize as razdel_tokenize


class Segmenter:
    def __init__(self, path):
        with open(path, 'r', encoding="utf-8") as file:
            self.vocab = set(word.strip().lower() for word in file if word.strip())

        self.tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru") 

        self._ascii_re = re.compile(r'^[\x00-\x7f]+$')
        self._punct_before_re = re.compile(r'\s+([.,!?;:])')
        self._open_after_re = re.compile(r'([“"\'«(\[{])\s+')
        self._close_before_re = re.compile(r'\s+([”"\'»)\]}])')
    
    def _is_ascii(self, token: str) -> bool:
        return bool(self._ascii_re.match(token))
        
    def _build_from_subtokens(self, token: str, subtokens: list[str]) -> list[str]:
        # склейка сабтокенов по словарю
        buffer = ""
        parts = list()
        for st in subtokens:
            piece = st.lstrip("#")
            if not piece: 
                continue
            buffer += piece
            if buffer.lower() in self.vocab:
                parts.append(buffer)
                buffer = ""
        if buffer == "":
            return parts # успешно разобран
        
        remained = buffer
        tmp = list()
        while remained:
            matched = False
            for l in range(len(remained), 0, -1):
                pref = remained[:l]
                if pref.lower() in self.vocab:
                    tmp.append(pref)
                    remained = remained[l:]
                    matched = True
                    break
            if not matched:
                return None
        parts.extend(tmp)
        return parts
    
    def preprocessor(self, text: str) -> str:
        # базовая сегментация razdel
        tokens = [t.text for t in razdel_tokenize(text)]
        text = " ".join(tokens)

        # удаление лишних пробелов перед знаками препинания и вокруг кавычек
        text = self._punct_before_re.sub(r"\1", text)
        text = self._open_after_re.sub(r"\1", text)
        text = self._close_before_re.sub(r"\1", text)

        return text

    def segment(self, text: str) -> str:
        tokens = text.split()
        out_tokens = list()

        for token in tokens:
            # англ / числа / пунктуация
            if self._is_ascii(token):
                out_tokens.append(token)
                continue
            
            # токенизация 
            subtokens = self.tokenizer.tokenize(token)
            parts = self._build_from_subtokens(token, subtokens) # попытка склейки слов по словарю
            if parts is not None:
                out_tokens.extend(parts)
                continue
            

            # восстановление строки при частично удачной или неудачной попытке склейки
            remained = token
            tmp = list()
            while remained:
                matched = False
                for l in range(len(remained), 0, -1):
                    pref = remained[:l]
                    if pref.lower() in self.vocab:
                        tmp.append(pref)
                        remained = remained[l:]
                        matched = True
                        break
                if not matched:
                    # сохраняем только остаток, а не весь токен
                    tmp.append(remained)
                    remained = ""
            out_tokens.extend(tmp)

        # финальная чистка
        s = " ".join(out_tokens)
        s = self._punct_before_re.sub(r"\1", s)
        s = self._open_after_re.sub(r"\1", s)
        s = self._close_before_re.sub(r"\1", s)

        return s
    
    def process(self, text: str) -> str:
        return self.segment(self.preprocessor(text))

model = Segmenter(path="words_cases.txt")

# для визуальной оценки
# for text in df.text_no_spaces:
#     restored = model.process(text)
#     print(restored)


restored = df['text_no_spaces'].apply(model.process)
make_submission(df, restored)

# Mean F1 = 59.659%